In [28]:
import warnings 
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
BASE, BU, TDFP, MinT = {},{},{},{}
nbase = ['lgb_base','ets_base']
nbu   = ['lgb_base','lgb_bu','ets_base','ets_bu']
ntdfp = ['lgb_base','lgb_td','ets_base','ets_td']
nmint = ['lgb_base','lgb_mint','ets_base','ets_mint']

for i in tqdm([1]):
    BASE_ = pd.concat([pd.read_pickle(f"721lgbInvtSim_L{i}.pkl"),
                       pd.read_pickle(f"721etsInvtSim_L{i}.pkl")])
    BU_   = pd.read_pickle(f"721BUOrder_L{i}.pkl")
    TDFP_ = pd.read_pickle(f"721TDFPOrder_L{i}.pkl")
    MinT_ = pd.read_pickle(f"721MinTOrder_wls-var_L{i}.pkl")
    
    BASE_ = BASE_[BASE_['name'].isin(nbase)]
    BU_   = BU_[BU_['name'].isin(nbu)]
    TDFP_ = TDFP_[TDFP_['name'].isin(ntdfp)]
    MinT_ = MinT_[MinT_['name'].isin(nmint)]
    
    BASE[i] = BASE_
    BU[i]   = BU_
    TDFP[i] = TDFP_
    MinT[i] = MinT_

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:07<00:00,  7.36s/it]


In [29]:
lp1 = {"l10" :(0,3049),"l11":(3049,3049+9147),"l12":(3049+9147,3049+9147+30490)}
lvls = []
#n = 1
for i in lp1.keys():
    a,b = lp1[i]
    lvls.append([i[1:]]*28*(b-a))
   # n+=1
levels = pd.concat([pd.DataFrame({'levels':lvls[i]}) for i in range(len(lvls))]).reset_index(drop=True)
path1  = 'C:/PycharmProjects/yjz/Extension for hts/JayCode/Models/'
test   = pd.read_pickle(f"{path1}721future_28.pkl").reset_index(drop=True)
# S       = pd.read_pickle(f"{path1}S_df.pkl")
tags   = pd.read_pickle(f"{path1}tags.bin")
ts     = pd.concat([levels, test[['unique_id','ds']]],axis = 1)
cate = pd.DataFrame({'group': [i[-3] for i in tqdm(ts['unique_id'].str.split('/'))]})
ts = pd.concat([ts, cate],axis = 1)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1195208/1195208 [00:00<00:00, 2145463.19it/s]


In [30]:
def lvl10_12(df, ts, names):
    empty = []
    df_ = df.copy()
    for i in names:
        dfe = df_[df_['name'] == i]
        dfe = pd.concat([ts,dfe],axis =1)
        dfe = dfe[dfe['levels'].isin(['10','11','12'])]
        #el = []
        #for i in ['FOODS','HOBBIES','HOUSEHOLD']:
            #e = dfe[dfe['unique_id'].str.contains(i)]
            #e['group'] = i
            #el.append(e)
        #empty.append(pd.concat(el).reset_index(drop = True))
        empty.append(dfe)
    return pd.concat(empty)
'''
def lvl10_12(df, ts, names):
    empty = []
    df_ = df.copy()
    for i in names:
        dfe = df_[df_['name'] == i]
        dfe = pd.concat([ts,dfe],axis =1)
        dfe = dfe[dfe['levels'].isin(['10','11','12'])]
        el = []
        for i in ['FOODS','HOBBIES','HOUSEHOLD']:
            e = dfe[dfe['unique_id'].str.contains(i)]
            e['group'] = i
            el.append(e)
        empty.append(pd.concat(el).reset_index(drop = True))
        #empty.append(dfe)
    return pd.concat(empty)'''

BASE2, BU2, TDFP2, MinT2 = {},{},{},{}
for i in tqdm([1]):
    BASE2[i] = lvl10_12(df = BASE[i], ts = ts, names = nbase)
    BU2[i]   = lvl10_12(df = BU[i]  , ts = ts, names = nbu)
    TDFP2[i] = lvl10_12(df = TDFP[i], ts = ts, names = ntdfp)
    MinT2[i] = lvl10_12(df = MinT[i], ts = ts[ts['levels'].isin(['10','11','12']).reset_index(drop=True)], names = nmint)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:15<00:00, 15.06s/it]


In [33]:
pd.DataFrame({1: [0,0,0,0]}).mean()

1    0.0
dtype: float64

In [5]:
BASE2[1]

,levels,unique_id,ds,group,name,true_demand,forecasts,ot_90,ip_90,net_90,...,net_95,backlog_95,ch_95,cb_95,ot_99,ip_99,net_99,backlog_99,ch_99,cb_99
0,10,TOTAL/FOODS/FOODS_1/FOODS_1_001,1914,FOODS,lgb_base,4,5.689794,7.338106,7.338106,7.338106,...,8.939324,0.0,8.939324,0.0,11.942942,11.942942,11.942942,0.0,11.942942,0.0
1,10,TOTAL/FOODS/FOODS_1/FOODS_1_001,1915,FOODS,lgb_base,5,4.679130,2.989336,5.327442,5.327442,...,6.928660,0.0,6.928660,0.0,2.989336,9.932278,9.932278,0.0,9.932278,0.0
2,10,TOTAL/FOODS/FOODS_1/FOODS_1_001,1916,FOODS,lgb_base,7,4.798928,5.119798,3.447240,3.447240,...,5.048458,0.0,5.048458,0.0,5.119798,8.052076,8.052076,0.0,8.052076,0.0
3,10,TOTAL/FOODS/FOODS_1/FOODS_1_001,1917,FOODS,lgb_base,1,5.980217,8.181289,10.628529,10.628529,...,12.229747,0.0,12.229747,0.0,8.181289,15.233366,15.233366,0.0,15.233366,0.0
4,10,TOTAL/FOODS/FOODS_1/FOODS_1_001,1918,FOODS,lgb_base,9,5.048564,0.068346,1.696875,1.696875,...,3.298093,0.0,3.298093,0.0,0.068346,6.301712,6.301712,0.0,6.301712,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195203,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1937,HOUSEHOLD,ets_base,0,0.076043,0.000000,0.557056,0.557056,...,0.692609,0.0,0.692609,0.0,0.000000,0.946883,0.946883,0.0,0.946883,0.0
1195204,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1938,HOUSEHOLD,ets_base,0,0.074525,0.000000,0.557056,0.557056,...,0.692609,0.0,0.692609,0.0,0.000000,0.946883,0.946883,0.0,0.946883,0.0
1195205,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1939,HOUSEHOLD,ets_base,0,0.074324,0.000000,0.557056,0.557056,...,0.692609,0.0,0.692609,0.0,0.000000,0.946883,0.946883,0.0,0.946883,0.0
1195206,12,TOTAL/WI/WI_3/HOUSEHOLD/HOUSEHOLD_2/HOUSEHOLD_...,1940,HOUSEHOLD,ets_base,0,0.073504,0.000000,0.557056,0.557056,...,0.692609,0.0,0.692609,0.0,0.000000,0.946883,0.946883,0.0,0.946883,0.0


In [26]:
np.array([]).mean()

nan

In [31]:
def dictgenerator(df):
    df_ = df.copy()
    cate = {}
    for i in ['FOODS','HOBBIES','HOUSEHOLD']:
        A = df_[df_['group'] == i]
        lp  = {}
        for j in df_['levels'].unique():
            lp[j] = (A[A['levels'] == j].index.min(), A[A['levels'] == j].index.max()+1)
        cate[i] = lp
    return cate
a = BASE2[1]
a = a[a['name'] == 'lgb_base']
lp = dictgenerator(a)
#lp
'''
def dictgenerator(df):
    df_ = df.copy()
    cate = {}
    for i in ['FOODS','HOBBIES','HOUSEHOLD']:
        A = df_[df_['group'] == i]
        lp  = {}
        for j in df_['levels'].unique():
            lp[j] = (A[A['levels'] == j].index.min(), A[A['levels'] == j].index.max()+1)
        cate[i] = lp
    return cate
a = BASE2[1]
a = a[a['name'] == 'lgb_base']
lp = dictgenerator(a)'''
lp

{'FOODS': {'10': (0, 40236), '11': (85372, 296352), '12': (341488, 1150072)},
 'HOBBIES': {'10': (40236, 56056),
  '11': (125608, 312172),
  '12': (381724, 1165892)},
 'HOUSEHOLD': {'10': (56056, 85372),
  '11': (141428, 341488),
  '12': (397544, 1195208)}}

In [20]:
40236/28

1437.0

In [32]:
import warnings 
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
'''
def check(lp:dict, df:pd.DataFrame,col: str, gap:int = 28, back_ = True):
    asl = []
    for i in lp.keys():
        l1 = np.array(df[col][lp[i][0]:lp[i][1]])
        l2 = np.array(df['true_demand'][lp[i][0]:lp[i][1]])
        if back_:
            asl2 = []
            for j in range(len(l1)):
                if l1[j] == 0 or l2[j] == 0:
                    a = 1 #if  else 
                else:
                    if (l1[j]/l2[j]) <= 1:
                        a = 1-(l1[j]/l2[j])
                    else:
                        a = 0
                asl2.append(a)
            asl.append(np.mean(asl2))
        else:
            asl.append(l1.mean())
    return asl
'''
def check(lp:dict, df:pd.DataFrame,col: str, gap:int = 28, back_ = True):
    asl = []
    for i in lp.keys():
        l1 = np.array(df[col][lp[i][0]:lp[i][1]])#*gap
        l2 = np.array(df['true_demand'][lp[i][0]:lp[i][1]])#*gap
        if back_:
            asl2 = []
            for j in range(len(l1)):
                if l1[j] == 0:
                    a = 1 #if  else 
                else:
                    a = 0
                asl2.append(a)
            asl.append(np.mean(asl2))
        else:
            asl.append(l1.mean())
    return asl
def levels_summary2(dff:pd.DataFrame, 
                   col_1:str, 
                   col_2:str,
                   col_3:str,
                   lp_:dict, 
                   name_:str,
                   tsl_: float,
                   senario_:str,
                   c = True , 
                   gap_:int = 28, 
                   lvls:int = 3, lvls_list = [10,11,12]):
    dff_                           = pd.DataFrame()
    dff_['Levels']                 = lvls_list
    dff_['Forecast_Methods']       = [name_]*lvls
    dff_['Target_Service_Level']   = [tsl_]*lvls
    dff_['Senario']                = [senario_]*lvls
    dff_['Achieved_Service_Level'] = check(lp = lp_, df = dff, col = col_1, back_ = c, gap = gap_)
    dff_['Holding_Costs']          = check(lp = lp_, df = dff, col = col_2, back_ = not c, gap = gap_)
    dff_['Backlogging_Costs']      = check(lp = lp_, df = dff, col = col_3, back_ = not c, gap = gap_)
    return dff_
def asl_chcb(lp:dict, Idf:pd.DataFrame, names, names_m, col_list3, senario:str, lvls_ = 3, lvls_list_ = [10,11,12]):
    summary_list = []
    for i in tqdm(range(len(names))):
        idf = Idf[Idf['name'] == names[i]]
        for j in col_list3:
            summary_list.append(levels_summary2(lp_ = lp, 
                                            dff= idf, 
                                            name_ = names_m[i],
                                            col_1 = j[0], 
                                            col_2 = j[1], 
                                            col_3 = j[2], 
                                            tsl_ = j[3], senario_ = senario, lvls = lvls_, lvls_list = lvls_list_))
    return pd.concat(summary_list)

In [33]:
def collect(cate:str, BASE, BU, TDFP, MinT,lp):
    columns = ["name",'true_demand',"backlog_90","ch_90","cb_90","backlog_95","ch_95","cb_95","backlog_99","ch_99","cb_99"]
    col_list3 = [('backlog_90','ch_90','cb_90', '90%'), 
             ('backlog_95','ch_95','cb_95', '95%'), 
             ('backlog_99','ch_99','cb_99', '99%')]
    lp_ = lp[cate]
    names   = ['lgb_base',  'ets_base']
    names_m = ['LGBM','ETS']
    #BASE    = BASE
    #BASE   = BASE[BASE['group'] == cate]
    L_BASE  = asl_chcb(lp=lp_, Idf = BASE, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BASE')
    
    names   = ['lgb_base', 'lgb_bu',  'ets_base', 'ets_bu']
    names_m = ['LGBM','LGBM_BU','ETS','ETS_BU']
    #BU      = BU
    #BU     = BU[BU['group'] == cate]
    L_BU    = asl_chcb(lp=lp_, Idf = BU, names = names, names_m = names_m, col_list3 = col_list3, senario = 'BU_IR')
    
    names   = ['lgb_base','lgb_td', 'ets_base','ets_td']
    names_m = ['LGBM','LGBM_TD','ETS','ETS_TD']
    #names     = ['lgb_td', 'ets_td']#'lgb_td',,'ets_td'
    #names_m   = ['LGBM_TD','ETS_TD']#'LGBM_TD','ETS_TD'
    #TDFP   = TDFP
    #TDFP    = TDFP[TDFP['group'] == cate]
    L_TDFP  = asl_chcb(lp=lp_, Idf = TDFP, names = names, names_m = names_m, col_list3 = col_list3, senario = 'TD_IR')
    
    names   = ['lgb_base', 'lgb_mint', 'ets_base', 'ets_mint']
    names_m = ['LGBM','LGBM_MinT','ETS','ETS_MinT']
    #MinT   = MinT
    #MinT    = MinT[MinT['group'] == cate]
    L_MinT  = asl_chcb(lp=lp_, Idf = MinT, names = names, names_m = names_m, col_list3 = col_list3, senario = 'MinT_IR')
    all_ = pd.concat([L_BASE,L_BU,L_TDFP,L_MinT])
    return all_#mint_ols
################################ FOODS:
method = 'wls-var'
for lead_time in [1]:
    print(f"Lead_time is {lead_time}")
    empty = []
    for j in ['FOODS','HOBBIES','HOUSEHOLD']:
        print(f"The group is: {j}")
        all_ = collect(lp = lp,cate = j, BASE= BASE2[lead_time], BU=BU2[lead_time], TDFP=TDFP2[lead_time], MinT=MinT2[lead_time])
        all_['Group'] = j
        empty.append(all_)
    pd.concat(empty).to_csv(f"721Base2IR_L{lead_time}_Category.csv", index = False)

Lead_time is 1
The group is: FOODS


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:06<00:00,  1.54s/it]


The group is: HOBBIES


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.36s/it]


The group is: HOUSEHOLD


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:05<00:00,  1.39s/it]


In [27]:
a = pd.read_csv("721Base2IR_L1_Category.csv")
a#["Senario"].unique()

,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs,Group
0,10,LGBM,90%,BASE,0.868501,8.501712,4.941229,FOODS
1,11,LGBM,90%,BASE,0.882430,3.808770,2.370622,FOODS
2,12,LGBM,90%,BASE,0.898742,1.985792,1.204012,FOODS
3,10,LGBM,95%,BASE,0.910553,10.628379,6.655202,FOODS
4,11,LGBM,95%,BASE,0.918556,4.723445,3.380122,FOODS
...,...,...,...,...,...,...,...,...
319,11,ETS_MinT,95%,MinT_IR,0.921953,4.128103,2.825406,HOUSEHOLD
320,12,ETS_MinT,95%,MinT_IR,0.864918,1.626348,3.133193,HOUSEHOLD
321,10,ETS_MinT,99%,MinT_IR,0.991097,13.379181,2.350686,HOUSEHOLD
322,11,ETS_MinT,99%,MinT_IR,0.959897,5.703880,7.478763,HOUSEHOLD


In [15]:
b = pd.concat(empty)
c = b[b['Forecast_Methods'] == 'LGBM']
c[c['Senario'] == 'BASE']

,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs,Group
0,10,LGBM,90%,BASE,0.868501,8.501712,4.941229,FOODS
1,11,LGBM,90%,BASE,0.882430,3.808770,2.370622,FOODS
2,12,LGBM,90%,BASE,0.898742,1.985792,1.204012,FOODS
0,10,LGBM,95%,BASE,0.910553,10.628379,6.655202,FOODS
1,11,LGBM,95%,BASE,0.918556,4.723445,3.380122,FOODS
2,12,LGBM,95%,BASE,0.926485,2.423002,1.827545,FOODS
0,10,LGBM,99%,BASE,0.957799,14.798471,15.234703,FOODS
1,11,LGBM,99%,BASE,0.958451,6.513202,8.817886,FOODS
2,12,LGBM,99%,BASE,0.958675,3.272476,5.338997,FOODS
0,10,LGBM,90%,BASE,0.878698,4.912053,2.848906,HOBBIES


In [13]:
c[c['Senario'] == 'TD_IR']

,Levels,Forecast_Methods,Target_Service_Level,Senario,Achieved_Service_Level,Holding_Costs,Backlogging_Costs,Group
0,10,LGBM,90%,TD_IR,0.979014,8.501712,4.941229,FOODS
1,11,LGBM,90%,TD_IR,0.923020,2.748666,4.942178,FOODS
2,12,LGBM,90%,TD_IR,0.866913,1.082001,4.149312,FOODS
0,10,LGBM,95%,TD_IR,0.986852,10.628379,6.655202,FOODS
1,11,LGBM,95%,TD_IR,0.940538,3.267238,8.221288,FOODS
2,12,LGBM,95%,TD_IR,0.880831,1.212168,7.827040,FOODS
0,10,LGBM,99%,TD_IR,0.994401,14.798471,15.234703,FOODS
1,11,LGBM,99%,TD_IR,0.963178,4.309118,27.622646,FOODS
2,12,LGBM,99%,TD_IR,0.903668,1.470413,33.001251,FOODS
0,10,LGBM,90%,TD_IR,0.976619,4.912053,2.848906,HOBBIES
